In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex, rounding     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '90'
formatted_attribute = 'width'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2426

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [5]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Width [Nom]', 'Width']
df_cur = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df_cur))

1906


In [6]:
df_cust=df_cur[(df_cur['Width [Nom]'].astype(str)!='[]')&(df_cur['Width [Nom]'].astype(str)!='NaN')&(df_cur['Width [Nom]'].astype(str)!='nan')&(df_cur['Width [Nom]'].astype(str)!='Assorted')&(df_cur['Width [Nom]'].astype(str)!='N/A')&(df_cur['Width [Nom]'].astype(str)!='Letter')    ]
# length[(length['Length'].astype(str)!='[]')&(length['Length'].astype(str)!='nan')&(length['Length'].astype(str)!='NaN')]
df_cust['match']=df_cust['Width [Nom]'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).apply(lambda x: re.sub(r'\"',' in',str(x))).apply(lambda x: re.sub(r"\'",'"',str(x)))
df_cust['match'].explode().value_counts()
df_cust[curation_col]=df_cust['match']
rounding(df_cust, curation_col,'a-eghj-mo-su-z')

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Width [Nom], Width, match, Q:width, rounding]
Index: []

In [7]:
matchcust=df_cust[['external_id',curation_col]]
print(len(matchcust))

15


# Find more values

In [8]:
df_cur=df_cur[(df_cur['Width [Nom]'].astype(str)=='[]')|(df_cur['Width [Nom]'].astype(str)=='NaN')|(df_cur['Width [Nom]'].astype(str)=='nan')|(df_cur['Width [Nom]'].astype(str)=='Assorted')]

In [9]:
pat=r'(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(pat, x))
regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_width=r'(?i)(\d+(?:.\d\/\d+|\.\d+|\/\d)?[\W]{0,2}Wi?d?t?h?e?(?!eigh)\b)|()'
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|()"
df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d|()'
df_cur['new_matches'] = df_cur['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [10]:
print(len(df_cur))
pat=r'''(?i)(\d+(?:.?\d+\/\d+|\.\d+)?(?:"|'|'')?\s?x)|(\d+\/\d+(?:"|'|'')?\s?x)|(\d+\'\d+"\s?x)|()'''
df_cur['match'] = df_cur['blank_matches'].apply(lambda x: re_extract(pat, str(x)))


pap=df_cur[df_cur['match'].astype(str)!='[]']
print(len(pap))
pap['matchez']=pap['match'].apply(lambda x: natsorted(x))#.apply(lambda x: sorted(x))
pap['matchez']=pap['matchez'].apply(lambda x: re.sub(r'"','in',str(x))).apply(lambda x: re.sub(r"''",' in',str(x))).apply(lambda x: re.sub(r'(?i)\s?x\s?','',str(x))).apply(lambda x: re.sub(r"\[\'|\[in",'["',str(x))).apply(lambda x: re.sub(r"\'\]|in\]",'"]',str(x))).apply(lambda x: re.sub(r"\'\s?,\s?\'",'","',str(x))).apply(lambda x: re.sub(r"\'",' ft',str(x)))
pap['matchez']=pap['matchez'].apply(lambda x: re.sub(r'in',' in',str(x))).apply(lambda x: re.sub(r'(?<!a-z)\"\]',' in"]',str(x))).apply(lambda x: re.sub(r'ft in','ft',str(x))).apply(lambda x: re.sub(r'in in','in',str(x))).apply(lambda x: re.sub(r'(?<![a-z])","',' in","',str(x))).apply(lambda x: re.sub(r'\[\[','[',str(x))).apply(lambda x: re.sub(r'\]\]',']',str(x)))#.apply(lambda x: re.sub(r'1000','1,000',str(x)))                      
pap['matchez']=pap['matchez'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])


pap['matchea']=pap['matchez'].apply(lambda x: sorted(x))#.apply(lambda x: natsorted(x))
pap[curation_col]=pap['matchea'].apply(lambda x: re.sub(r"\[\'\[",'[',str(x))).apply(lambda x: re.sub(r"\]\'\]",']',str(x)))
rounding(pap, curation_col,'a-eghj-mo-su-z')
pap=pap[pap['rounding'].astype(str)=='[]']
rounding(pap, curation_col,'a-eghj-mo-su-z')

1891
559


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Width [Nom], Width, matches, matches_length, matches_height, matches_width, matches_diameter, matches_ft, new_matches, blank_matches, blank_matches_long, match, matchez, matchea, Q:width, rounding]
Index: []

In [41]:
matchpaperfour=pap[['external_id',curation_col]]
# pap['matchez'].explode().value_counts()

In [12]:
# df_more=df[(df['matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['matches_width'].astype(str)!='[]')]
# print(len(df_more))
# # df_more

In [13]:
# df_more['matches_more']=df_more['matches_width'].apply(lambda x: re.sub(r'''(?i)\\" W|".?W|''.?W|”.?W|˝.?W|”.?W|” w|″ W''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)\\' W|'.?W|'.?W''',' ft',str(x))).apply(lambda x: re.sub(r'(?i)\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# df_more['matches_more']=df_more['matches_more'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

In [14]:
# df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub('ft in','ft',str(x)))                                                                           
# df_more['matches_more']=df_more['matches_more'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_more['matches_more']=df_more['matches_more'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

# df_more[curation_col]=df_more['matches_more']
# matchesmore=df_more[['external_id',curation_col]]
# print(len(matchesmore))

In [15]:
# df_na=df[(df['matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['matches_width'].astype(str)=='[]')]
# df_na[curation_col]='n/a'
# matches_na=df_na[['external_id',curation_col]]
# # matches_na

# NA

In [16]:
df_na=df_cur[(df_cur['matches_length'].astype(str)=='[]')&(df_cur['matches_height'].astype(str)=='[]')&(df_cur['matches_width'].astype(str)=='[]')&(df_cur['matches_ft'].astype(str)=='[]')&(df_cur['matches_diameter'].astype(str)=='[]')&(df_cur['blank_matches'].astype(str)=='[]')&(df_cur['blank_matches_long'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
df_na[curation_col]='n/a'
print(len(df_na))
matches_na=df_na[['external_id',curation_col]]
# df_na

240


In [17]:
# # Two
# (?i)(?<!x)\W\W\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*\WI?n?c?h?"?'?\W?[lwdh]?\W?\W?\W?\W(?!x)
#  #Three
# |(\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?)
# # All
# (\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x?.?\d*.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?)

In [18]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?([lwdh]\b)?)|()'''
# pat_split_THREE=r'''(?i)(\W?\W?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d*.?\d+\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?\w?)|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*.{0,4}x\"\d+\.?\d*.{0,4}"\])|()'''

In [19]:
# (?i)((\d+(\.\d+|\W\d+\/d*)?).?\w{0,4}\'?\'?\"?.?\W?[lwhd]?\s?x.?(\d+(\.\d*|\W\d+\/d*)?).?\w{0,4}\'?\'?\"?.?\W?[lwhd]?\b\s?)

In [20]:
mm=r'''(?i)(\d+.?mm(?!\/sec)|\d+.?mm(?!\/sec))|()'''
df_cur['mm']=df_cur['long_desc'].apply(lambda x: re_extract(mm, str(x)))

print(len(df_cur[df_cur['mm'].astype(str)=='[]']))
print(len(df_cur[df_cur['mm'].astype(str)!='[]']))

df_mm=df_cur[df_cur['mm'].astype(str)!='[]']
df_not_mm=df_cur[df_cur['mm'].astype(str)=='[]']
# df_mm

1870
21


# Min Max (Min is width)

In [21]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?([lwdh]\b)?)|()'''
# pat_split_THREE=r'''(?i)(\[\"(\d+(\.\d*|\W\d+\/d*)?).?\w{0,4}.?x.?(\d+(\.\d*|\W\d+\/d*)?).?\w{0,4}.?x.?(\d+(\.\d*|\W?\d+\/d*)?).?\w{0,4}\"\])|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*\W{0,4}x\"\d+\.?\d*\W{0,4}"\])|()'''

# df_not_mm['matches_split']=df_not_mm['product_name'].apply(lambda x: re_extract(all_split, str(x)))
# df_not_mm['matches_split_frac']=df_not_mm['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# df_not_mm['matches_split_frac']=df_not_mm['matches_split_frac'].apply(lambda x: re.sub(r'''�''','',str(x))).apply(lambda x: re.sub(r'''(\[\"\[\')|(\[\'\[\")|(\[\"\[\\\')''',"['[\'",str(x))).apply(lambda x: re.sub(r'''(\'\]\"\])|(\\\'\]\"\])|(\"\]\'\])''','''\']']''',str(x))).apply(lambda x: re.sub(r'''\"''',' in',str(x))).apply(lambda x: re.sub(r'''(\[\'\[\\\')|(\[\'\[\')''','["',str(x))).apply(lambda x: re.sub(r'''(\\\'\]\'\])|(\'\]\'\])''','"]',str(x))).apply(lambda x: re.sub(r'''(?i)x''','"x"',str(x))).apply(lambda x: re.sub(r'''\'''',' ft',str(x))).apply(lambda x: re.sub(r''' ''','',str(x))).apply(lambda x: re.sub(r'''\\ft\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''X''','x',str(x))).apply(lambda x: re.sub(r'''in''',' in',str(x))).apply(lambda x: re.sub(r'''ft''',' ft',str(x))).apply(lambda x: re.sub(r'''\,.*''','"]',str(x))).apply(lambda x: re.sub(r'''\\\\.?ft''','ft',str(x)))                                                                                                                                                             

# df_not_mm['matches_split_three'] = df_not_mm['matches_split_frac'].apply(lambda x: re_extract(pat_split_THREE, str(x)))
# df_not_mm['matches_split_two'] = df_not_mm['matches_split_frac'].apply(lambda x: re_extract(pat_split_TWO, str(x)))


# regex_pattern_blank=r'(?i)(\d+\.\d*.?w\b)|()'
# df_not_mm['matches'] = df_not_mm['matches_split_frac'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# split=df_not_mm[df_not_mm['matches_split'].astype(str)!='[]']
# print(len(split))
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# print(len(split[split['matches_split_three'].astype(str)!='[]']))

In [22]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?w\b)|()'''
# split['matches']=split['matches_split_frac'].apply(lambda x: re_extract(pat_call_cout, str(x)))

# print(len(split[split['matches'].astype(str)!='[]']))
# width_call_out=split[split['matches'].astype(str)!='[]']
# width_call_out['matches']=width_call_out['matches'].apply(lambda x: re.sub(r'''(?i)w''',' in',str(x))).apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''(')''','',str(x)))
# width_call_out[curation_col]=width_call_out['matches']
# matcheswidthcallout=width_call_out[['external_id',curation_col]]

In [23]:
# matcheswidthcallout

In [24]:
# del split['new_matches']
# del split['matches_length']
# del split['matches_height']
# del split['matches_width']
# del split['matches_diameter']
# del split['matches_ft']
# del split['blank_matches']
# del split['blank_matches_long']

In [25]:
# # split['matches_split_two'].explode().value_counts()[0:500]
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# split_two=split[(split['matches_split_two'].astype(str)!='[]')&(split['matches'].astype(str)=='[]')]
# print(len(split_two))

In [26]:
# pat_split_width_inch=r'''(?i)(ft)|()'''
# split_two['ft']=split_two['matches_split_two'].apply(lambda x: re_extract(pat_split_width_inch, str(x)))
# split_two_inch=split_two[split_two['ft'].astype(str)=='[]']
# print(len(split_two_inch))
# split_two_inch.head(2)

In [27]:
# del split_two_inch['mm']
# del split_two_inch['ft']

In [28]:
# pat_split_length=r'''(?i)(x\"\d+\.?\d*.{0,4}"\])|()'''
# pat_split_width=r'''(?i)(\[\"\d+\.?\d*.{0,4}x)|()'''
# split_two_inch['length']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_length, str(x)))
# split_two_inch['width']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_width, str(x)))

In [29]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# # split_two_inch

In [30]:
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''12.5di''','12.5',str(x))).apply(lambda x: re.sub(r'''10\.5\?''','10.5',str(x))).apply(lambda x: re.sub(r'''0\.62\-''','0.62',str(x))).apply(lambda x: re.sub(r'''18\-''','18',str(x))).apply(lambda x: re.sub(r'''15\)''','15',str(x)))                                 
# split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'\?|\-| ','',str(x)))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'\?|\-| ','',str(x)))

In [31]:
# pat_call_cout=r'''(?i)(\d+\.?\d*.?.?[a-z]\b)|()'''
# split_two_inch['matches']=split_two_inch['length'].apply(lambda x: re_extract(pat_call_cout, str(x)))
# split_two_inch=split_two_inch[split_two_inch['matches'].astype(str)=='[]']

In [32]:
# split_two_inch[split_two_inch['length'].astype(str)=='']

In [33]:
# split_two_inch['length'].explode().value_counts()
# split_two_inch[split_two_inch['length'].astype(str)=='15)']

In [34]:
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

In [35]:
# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [36]:
# split_two_inch=split_two_inch[split_two_inch['length'].astype(str)!=""]
# split_two_inch=split_two_inch[split_two_inch['width'].astype(str)!=""]
# split_two_inch['length']=split_two_inch['length'].apply(lambda x:re.sub(r' in','',str(x)))#.apply(lambda x:str(f'"{x}"'))
# split_two_inch['width']=split_two_inch['width'].apply(lambda x:re.sub(r' in','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split_two_inch['length']=split_two_inch['length'].astype(float)
# split_two_inch['width']=split_two_inch['width'].astype(float)

# split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
# split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
# split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# split_two_inch['Min']=split_two_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [37]:
# split_two_inch[curation_col]=split_two_inch['Min']
# msplit=split_two_inch[['external_id',curation_col]]
# msplit[curation_col]=split_two_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))

In [38]:
# split_two_inch[500:1000]

In [39]:
print(len(df_cur))
print(len(matches_na))

1891
240


In [42]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matches_natural))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_na))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matches_natural))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matches_width))

matches_na
matchcust


In [ ]:
stop

# send to the folder for upload

In [45]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, df, matchcust)

In [46]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df_na, matches_na)

In [50]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
# looks_good('WBMason', attribute, df, matchcustom)

In [51]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchpaperfour)